In [103]:
import dotenv
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from typing import List

import pandas as pd

dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("GPT_KEY")

# https://python.langchain.com/docs/expression_language/get_started/


MODEL = ChatOpenAI(model="gpt-4")


In [104]:
conversa_id = 2
roteiro_id = 2

In [105]:
# df_messages é o dataframe com as mensagens referentes a conversa atual  
df_messages = pd.read_csv("data/conversation.csv", sep=";", encoding="UTF-8")

# df_script é o dataframe com os roteiros
df_script = pd.read_csv("data/roteiros.csv", sep=";", encoding="UTF-8")

# df_script_stages é o dataframe com as etapas dos roteiros
df_script_stages = pd.read_csv("data/roteiro_stages.csv", sep=";", encoding="UTF-8")

# pegamos o contexto, ou seja, todas as mensagens trocadas até agora na conversa
context = df_messages[df_messages['CONVERSATION_ID'] == conversa_id]

# se o contexto é vazio, pegamos a primeira mensagem do roteiro
if len(context) == 0:
        first_message = df_script_stages[(df_script_stages['ROTEIRO_ID'] == roteiro_id) & (df_script_stages['STAGE'] == 0)]
        first_message = {
            "CONVERSATION_ID" : [conversa_id], 
            "ROTEIRO_ID" : [roteiro_id], 
            "STAGE" : [0], 
            "MENSAGEM_ID" : [1], 
            "TRANSCRIPT" : [first_message['OPTION'].values[0]], 
            "SENDER": ["CHAT"]
        }
        df_messages = pd.concat([df_messages, pd.DataFrame(first_message)], ignore_index=True)
        df_messages.to_csv('data/conversation.csv', index=False, sep=";")
        context = df_messages[df_messages['CONVERSATION_ID'] == conversa_id]

In [106]:
df_script_stages

,ROTEIRO_ID,STAGE,OPTION
0,1,0,"Doutor, boa tarde. Eu sou a Sandra, e vim rece..."
1,1,1,Eu sou contra relacionamentos abertos pois ele...
2,1,1,Eu não tenho nenhum tipo de contato com pessoa...
3,1,1,Eu consigo assobiar e chupar cana ao mesmo tempo
4,2,0,"Vamos começar a entrevista? Primeiramente, com..."
5,2,1,Isso não faz sentido algum! Essa estrutura de ...
6,2,1,"Sua resposta foi mais ou menos, mas não inclui..."
7,2,1,"Perfeito, você acertou! A estrutura de dados H..."


In [107]:
context

,CONVERSATION_ID,ROTEIRO_ID,STAGE,MENSAGEM_ID,TRANSCRIPT,SENDER
0,2,2,0,1,"""Vamos começar a entrevista? Primeiramente, c...",CHAT


In [108]:
df_messages

,CONVERSATION_ID,ROTEIRO_ID,STAGE,MENSAGEM_ID,TRANSCRIPT,SENDER
0,2,2,0,1,"""Vamos começar a entrevista? Primeiramente, c...",CHAT


In [109]:
df_script = df_script[df_script['ROTEIRO_ID'] == roteiro_id]

In [110]:
df_script

,ROTEIRO_ID,CONTEXT,CHAT,USER
1,2,Você deve participar de uma entrevista de prog...,Entrevistador,Entrevistado


In [111]:
def buildPrompt(context: pd.DataFrame, df_scripts: pd.Series, message:str, chat_role:str):

    prompt = "O contexto da Atividade é:" + df_scripts["CONTEXT"] + "\n" + f"Você deve atuar como o {chat_role}" + "\n\n Essa é a conversa até agora: \n"

    messages = context['TRANSCRIPT'].to_list()
    senders = context['SENDER'].to_list()


    for i in range(len(context)):
        prompt += df_scripts[senders[i]] + " - " + messages[i] + "\n\n"

    prompt += "A última mensagem do/a " + df_scripts["USER"] + " foi:\n" + message

    prompt+= '''
Qual seria a próxima resposta mais apropriada, entre as seguintes:

    {Resposta1}

    {Resposta2}

    {Resposta3}

Responda somente com um número, correspondente a resposta escolhida
    '''

    return prompt

In [112]:
user_response = "Um Hashmap é uma estrutura de dados que funciona como uma lista, porém cada item da lista tem um ponteiro que aponta para o próximo item da lista"

In [113]:
chat_role = df_script["CHAT"].values[0]
prompt = buildPrompt(context=context, df_scripts=df_script.iloc[0], message=user_response, chat_role=chat_role)

In [114]:
prompt

'O contexto da Atividade é:Você deve participar de uma entrevista de programação\nVocê deve atuar como o Entrevistador\n\n Essa é a conversa até agora: \nEntrevistador -  "Vamos começar a entrevista? Primeiramente, como funciona a estrutura HASHMAP?"\n\nA última mensagem do/a Entrevistado foi:\nUm Hashmap é uma estrutura de dados que funciona como uma lista, porém cada item da lista tem um ponteiro que aponta para o próximo item da lista\nQual seria a próxima resposta mais apropriada, entre as seguintes:\n\n    {Resposta1}\n\n    {Resposta2}\n\n    {Resposta3}\n\nResponda somente com um número, correspondente a resposta escolhida\n    '

In [115]:
stage = context['STAGE'].max()+1
m_id = context['MENSAGEM_ID'].max()

In [116]:
stage

1

In [117]:
m_id

1

In [120]:
df_script_stages = df_script_stages[(df_script_stages['ROTEIRO_ID'] == roteiro_id) & (df_script_stages['STAGE'] == stage)]
options = df_script_stages['OPTION'].astype(str).to_list()

In [124]:
options

['Isso não faz sentido algum! Essa estrutura de dados não tem nada a ver com um Hashmap',
 'Sua resposta foi mais ou menos, mas não incluiu todas as informações relevantes: a estrutura de dados Hashmap é uma estrutura de dados que associa chaves de pesquisa a valores',
 'Perfeito, você acertou! A estrutura de dados Hashmap é uma estrutura de dados que associa chaves de pesquisa a valores']

In [121]:
def getResponse(options: List[str], prpt: str) -> int:

    prompt = ChatPromptTemplate.from_template(prpt)
    output_parser = StrOutputParser()

    chain = prompt | MODEL | output_parser

    n = chain.invoke({f"Resposta{i+1}":options[i] for i in range(len(options))})

    return int(n)

In [125]:
response = getResponse(options=options, prpt=prompt)

In [126]:
response

2

In [127]:
data = {
    "CONVERSATION_ID" : [conversa_id, conversa_id], 
    "ROTEIRO_ID" : [roteiro_id, roteiro_id], 
    "STAGE" : [stage, stage], 
    "MENSAGEM_ID" : [m_id+1, m_id+2], 
    "TRANSCRIPT" : [user_response, options[response-1]], 
    "SENDER": ["USER", "CHAT"]
}


user_and_chat_answers = pd.DataFrame(data)
df_messages = pd.concat([df_messages, user_and_chat_answers], ignore_index=True)
df_messages.to_csv('data/conversation.csv', index=False, sep=";")